<h1>Building a postgres database for crime reports</h1>
<p>A database named crimes_db, with a table: boston_crimes for storing data from a csv file</p>

In [10]:
import psycopg2
import csv
conn = psycopg2.connect("dbname=dq user=dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db OWNER dq")
conn.autocommit = False
conn.close()
conn = psycopg2.connect("dbname = crime_db user=dq")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [3]:
import pandas as pd
#get unique values from a column
def get_col_set(csv_filename, col_index):
    my_csv = pd.read_csv(csv_filename)
    return my_csv.iloc[:, col_index].unique()

#get number of unique values of columns in boston.csv
for i in range(0,7):
    print(len(get_col_set("boston.csv", i)))

298329
219
239
1177
7
18177
18177


In [4]:
#Get maximum length of any value in the description column
print(col_headers)
print(len(str((get_col_set("boston.csv", 6)))))
print(get_col_set("boston.csv", 5))


['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
84
[42.35779134 42.30682138 42.34658879 ... 42.346473   42.33987101
 42.3621837 ]


<p>Enum is used for day of the week as there are only 7 values. Integer is the apropriate size for the primary key, date is a perfect fit for the date column, and varchars of appropriate length are used for the remaining columns</p>

In [5]:
cur.execute("CREATE TYPE weekday AS ENUM ('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday');")
cur.execute("CREATE TABLE crimes.boston_crimes (incident_number INTEGER PRIMARY KEY, offense_code VARCHAR(4), description VARCHAR(58), date DATE, day_of_the_week weekday, lat VARCHAR(78), long VARCHAR(84));")

In [6]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM StDIN WITH CSV HEADER;", f)

In [7]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

In [8]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

ProgrammingError: role "readonly" already exists


In [9]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [11]:
conn.commit()
conn.close()

In [15]:
#Tests
conn = psycopg2.connect("dbname = crime_db user=dq")
cur = conn.cursor()
cur.execute(""" 
    SELECT
        rolname,
        rolsuper,
        rolcreaterole,
        rolcreatedb,
        rolcanlogin
    FROM pg_roles
    WHERE rolname IN ('postgres', 'readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
print(cur.fetchall())
cur.execute("""
    SELECT 
        grantor,
        grantee,
        table_catalog,
        table_schema,
        table_name,
        privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
print(cur.fetchall())
conn.commit()
conn.close()

[('readonly', False, False, False, False), ('readwrite', False, False, False, False), ('data_analyst', False, False, False, True), ('data_scientist', False, False, False, True)]
[]
